In [1]:
import os
import urllib
import copy
from collections import defaultdict

import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Create dataset dir if not present
if 'Datasets' not in os.listdir():
    os.mkdir('Datasets')

In [3]:
# Download datasets

## Adult Dataset - Train
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", "./Datasets/AdultDS.csv");

## Adult Dataset - Test
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", "./Datasets/AdultDSTest.csv");

## Adult Dataset Column Names
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names", "./Datasets/AdultColNames.csv");

In [4]:
# Basic preprocessing

# Load Adult dataset
df_adult = pd.read_csv('./Datasets/AdultDS.csv', header=None)
df_adult_test = pd.read_csv('./Datasets/AdultDSTest.csv', skiprows=1, header=None)

# Load and process col names from Adult
df_adult_colnames = open('./Datasets/AdultColNames.csv', 'r').readlines()
df_adult_colnames = [c.split(':')[0] for c in df_adult_colnames[96:]] + ['label']

# Get a list of the categorical columns
df_adult_cat_cols = open('./Datasets/AdultColNames.csv', 'r').readlines()
df_adult_cat_cols = [c.split(':')[1][:11] != ' continuous' for c in df_adult_cat_cols[96:]] + [False]
df_adult_cat_cols = pd.Series(df_adult_colnames)[df_adult_cat_cols].tolist()

# Get a list of the numerical columns
df_adult_num_cols = list(set(df_adult_colnames)-set(df_adult_cat_cols+['label']))
        
# Assign col names to Adult dataset
df_adult.columns = df_adult_colnames
df_adult_test.columns = df_adult_colnames

# Make label equal for train and test
df_adult['label'] = df_adult['label'].map({' <=50K': 0, ' >50K': 1})
df_adult_test['label'] = df_adult_test['label'].map({' <=50K.': 0, ' >50K.': 1})

# Get shape of train since it will be merged with test to OHE
train_shape = df_adult.shape

# Merge both datasets
df_adult_full = pd.concat([df_adult, df_adult_test])

# Make OHE
df_adult_full = pd.get_dummies(df_adult_full)

# Assign to the train and test set
df_adult = df_adult_full.iloc[:train_shape[0],:]
df_adult_test = df_adult_full.iloc[train_shape[0]:,:]

# Delete merged dataset
del df_adult_full

column_order = list(df_adult.columns)

In [190]:
# Save dataset
df_adult.to_pickle('adult_train.pkl')
df_adult_test.to_pickle('adult_test.pkl')

pd.Series(df_adult_num_cols).to_pickle('adult_num_cols.pkl');

In [5]:
# Create a base model
model_adult = RandomForestClassifier(random_state=42, n_jobs=-1)

In [6]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
# Make simple grid-search
rf_random = RandomizedSearchCV(estimator = model_adult, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42)

In [8]:
# Run grid search
rf_random.fit(df_adult.drop(columns=['label']), df_adult['label'])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.7s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   1.7s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   1.7s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   5.4s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   5.4s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   5.2s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=6

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [9]:
# Assign the best model
model_adult = rf_random.best_estimator_

In [10]:
def calc_acc_auc(df, label):
    # Get list of predicted classes
    pred_class = model_adult.predict(df)
    # Get list of probabilities
    pred_proba = model_adult.predict_proba(df)
    # Calc Accuracy
    accuracy = accuracy_score(label, pred_class)
    # Calc AUC
    auc = roc_auc_score(label, pd.DataFrame(pred_proba)[1].tolist())

    print(f'Accuracy:{accuracy}\nACU:{auc}')

In [11]:
# Calculate Accuracy and AUC for train
print('For train:')
calc_acc_auc(df_adult.drop(columns=['label']), df_adult['label'])

For train:
Accuracy:0.9039341543564386
ACU:0.9669074323663392


In [12]:
# Calculate Accuracy and AUC for test
print('For test:')
calc_acc_auc(df_adult_test.drop(columns=['label']), df_adult_test['label'])

For test:
Accuracy:0.8641975308641975
ACU:0.9160132219522799


In [ ]:
# Save model
import pickle
# now you can save it to a file
with open('model_test.pkl', 'wb') as f:
    pickle.dump(model_adult, f)

In [115]:
# def cfnow(factual, feat_types, model, it_max=100, has_ohe=False):

In [139]:
def check_factual(factual):
    # Factual must be a pandas Series
    try:
        assert type(factual) == pd.Series
    except AssertionError:
        raise TypeError(f'Factual must be a Pandas Series. However it is {type(factual)}.')

In [140]:
def check_vars(factual, feat_types):
    # The number of feat_types must be the same as the number of factual features
    try:
        missing_var = list(set(factual.index)-set(feat_types.keys()))
        extra_var = list(set(feat_types.keys())-set(factual.index))
        assert len(missing_var) == 0 and len(extra_var) == 0
    except AssertionError:
        if len(missing_var) > 0 and len(extra_var) > 0:
            raise AssertionError(f"\nThe features:\n {','.join(missing_var)}\nmust have their type defined in feat_types.\
                                 \n\nAnd the features:\n {','.join(extra_var)}\nare not defined in the factual point")
        elif len(missing_var) > 0:
            raise AssertionError(f"The features:\n {','.join(missing_var)}\nmust have their type defined in feat_types.")
        elif len(extra_var) > 0:
            raise AssertionError(f"The features:\n {','.join(extra_var)}\nare not defined in the factual point.")

In [141]:
def check_prob_func(factual, model_predict_proba):
    # Test model function and get the classification of factual
    try:
        prob_fact = model_predict_proba(factual.to_frame().T)
    except Exception as err:
        raise Exception('Error when using the model_predict_proba function.')

In [142]:
def standardize_predictor(factual, model_predict_proba):
    # Convert the output of prediction function to something that can be treated

    # Check how it's the output of multiple
    prob_fact_multiple = model_predict_proba(pd.concat([factual.to_frame().T, factual.to_frame().T]))

    # mp1 always return the 1 class and [Num] or [Num, Num, Num]
    if str(prob_fact).isnumeric():
        # Result returns a number directly

        if len(np.array(prob_fact_multiple).shape) == 1:
            # Single: Num
            # Multiple: [Num, Num, Num]
            mp1 = lambda x: np.array([model_predict_proba(x)]) if x.shape[0] == 1 else np.array(model_predict_proba(x))
        else:
            # Single: Num
            # Multiple: [[Num], [Num], [Num]]
            index_1 = 0
            if len(np.array(prob_fact_multiple)[0]) == 2:
                index_1 = 1
            # This function gives an array containing the class 1 probability
            mp1 = lambda x: np.array([model_predict_proba(x)]) if x.shape[0] == 1 else np.array(model_predict_proba(x))[:, index_1]

    elif len(np.array(prob_fact).shape) == 1:
        if len(np.array(prob_fact_multiple).shape) == 1:
            # Single: [Num]
            # Multiple [Num, Num, Num]
            mp1 = lambda x: np.array(model_predict_proba(x))
        else:
            # Single: [Num]
            # Multiple [[Num], [Num], [Num]]
            index_1 = 0
            if len(np.array(prob_fact_multiple)[0]) == 2:
                index_1 = 1
            mp1 = lambda x: np.array(model_predict_proba(x))[:, index_1]
    else:
        # Single: [[Num]]
        # Multiple [[Num], [Num], [Num]]
        index_1 = 0
        if len(prob_fact[0]) == 2:
            index_1 = 1
        # This function gives an array containing the class 1 probability
        mp1 = lambda x: np.array(model_predict_proba(x))[:, index_1]
    
    return mp1

In [143]:
def get_ohe_params(factual, has_ohe):
    ohe_list = []
    ohe_indexes = []
    # if has_ohe:
    if has_ohe:
        prefix_to_class = defaultdict(list)
        for col_idx, col_name in enumerate(factual.index):
            col_split = col_name.split('_')
            if len(col_split) > 1:
                prefix_to_class[col_split[0]].append(col_idx)

        ohe_list = [idx_list for _, idx_list in prefix_to_class.items() if len(idx_list) > 1]
        ohe_indexes = [item for sublist in ohe_list for item in sublist]
    
    return ohe_list, ohe_indexes

In [144]:
def adjust_model_class(factual, mp1):
    # Define the cf try
    cf_try = copy.copy(factual).to_numpy()

    mp1c = mp1
    # Adjust class, it must be binary and lower than 0
    if mp1([cf_try])[0] > 0.5:
        mp1c = lambda x: 1-mp1(x)
        
    return mp1c

In [167]:
def super_sedc(factual, mp1c, feat_types, it_max, ohe_list, ohe_indexes):
    # For Categorical (binary) there's only one change, flipping 0->1 or 1->0
    # For Numerical we can increase 50% of input or decrease 50%

    # Define the cf try
    cf_try = copy.copy(factual).to_numpy()

    # Identify the indexes of categorical and numerical variables
    indexes_cat = np.where(np.isin(factual.index, [c for c, t in feat_types.items() if t=='cat']))[0]
    indexes_num = sorted(list(set([*range(len(factual))])-set(indexes_cat.tolist())))

    # Create identity matrixes for each type of variable
    arr_changes_cat_bin = np.eye(len(factual))[list(set(indexes_cat)-set(ohe_indexes))]
    arr_changes_cat_ohe = np.eye(len(factual))
    arr_changes_num = np.eye(len(factual))[indexes_num]

    iterations = 1
    cf_try_prob = mp1c(factual.to_frame().T)[0]

    tabu_list = []

    # Repeat until max iterations 
    while cf_try_prob <= 0.5 and iterations < it_max:
        # Changes
        # For categorical binary
        changes_cat_bin = arr_changes_cat_bin*(1-2*cf_try)
        # For categorical ohe
        changes_cat_ohe_list = []
        for ohe_group in ohe_list:
            changes_cat_ohe_list.append(arr_changes_cat_ohe[ohe_group] - (arr_changes_cat_ohe[ohe_group]*cf_try).sum(axis=0))
        if len(changes_cat_ohe_list) > 0:
            changes_cat_ohe = np.concatenate(changes_cat_ohe_list)
        else:
            changes_cat_ohe = []
        # For numerical up - HERE, NUMBERS WHICH ARE ZERO WILL REMAIN ZERO
        changes_num_up = arr_changes_num*0.5*cf_try
        # For numerical down - HERE, NUMBERS WHICH ARE ZERO WILL REMAIN ZERO
        changes_num_down = -copy.copy(changes_num_up)

        # Create changes array
        changes = np.concatenate([c for c in [changes_cat_bin, changes_cat_ohe, changes_num_up, changes_num_down] if len(c) > 0])

        # Create array with CF candidates
        cf_candidates = cf_try + changes

        # Calculate probabilities
        prob_cf_candidates = mp1c(cf_candidates)

        # Identify which index had the best performance towards objective, it will take the first best
        best_arg = np.argmax(prob_cf_candidates)

        # Update CF try
        cf_try = cf_try+changes[best_arg]

        # Update the score
        cf_try_prob = mp1c([cf_try])[0]
        print(cf_try_prob)

        # Update number of tries
        iterations += 1

        # Repeat process
    return cf_try

In [175]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def cfnow(factual, feat_types, model_predict_proba, it_max=100, has_ohe=False):
    
    # Make checks
    check_factual(factual)
    check_vars(factual, feat_types)
    check_prob_func(factual, model_predict_proba)
    
    # Generate standardized predictor
    mp1 = standardize_predictor(factual, model_predict_proba)
    
    # Correct class
    mp1c = adjust_model_class(factual, mp1)
    
    # Generate OHE parameters if it has OHE variables
    ohe_list, ohe_indexes = get_ohe_params(factual, has_ohe)
    
    # Generate CF using SEDC
    cf_try = super_sedc(factual, mp1c, feat_types, it_max, ohe_list, ohe_indexes)
    
    return cf_try

In [176]:
factual = df_adult_test.drop(columns=['label']).iloc[0]

In [177]:
# consider as default all columns as numerical
feat_types = {c: 'num' if c in df_adult_num_cols else 'cat' for c in list(df_adult.columns) if c != 'label'}

In [178]:
model_predict_proba = model_adult.predict_proba

In [179]:
it_max = 100

In [180]:
has_ohe = True

In [191]:
test1 = cfnow(factual, feat_types, model_predict_proba, it_max=100, has_ohe=False)

0.04718898716665681
0.09130976330316087
0.13694788412964
0.19333279217061275
0.2550939972087062
0.3056915795485748
0.3449086705244311
0.40143348963177183
0.45984244367101296
0.5295809273141039


In [183]:
test2 = cfnow(factual, feat_types, model_predict_proba, it_max=100, has_ohe=True)

0.04963643097165748
0.12506966319978785
0.18760668980068188
0.24766627214379697
0.30129057910658136
0.3802682156022782
0.4777896000679885
0.6545936259831946


In [110]:
for ohe_group in ohe_list:
    print(sum(cf_try[ohe_group]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [657]:
copy.copy(factual).to_numpy()

array([    25, 226802,      7,      0,      0,     40,      0,      0,
            0,      0,      1,      0,      0,      0,      0,      0,
            1,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      1,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      1,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            1,      0,      0,      0,      0,      1,      0,      0,
            0,      1,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      1,      0,      0])

In [658]:
cf_try.astype(int)

array([    37, 226802,     15,      0,      0,     40,      0,      0,
            0,      0,      1,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      1,      0,      0,      0,      0,      0,      0,
            0,      1,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            1,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      1,      0,      0,      1,      0,      0,
            0,      1,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      1,      0,      0])

In [127]:
type(prob_fact)

numpy.ndarray